<table style="width:100%">
<tr>
<td style="vertical-align:middle; text-align:left;">
<font size="2">
Supplementary code for the <a href="http://mng.bz/orYv">Build a Large Language Model From Scratch</a> book by <a href="https://sebastianraschka.com">Sebastian Raschka</a><br>
<br>Code repository: <a href="https://github.com/rasbt/LLMs-from-scratch">https://github.com/rasbt/LLMs-from-scratch</a>
</font>
</td>
<td style="vertical-align:middle; text-align:left;">
<a href="http://mng.bz/orYv"><img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/cover-small.webp" width="100px"></a>
</td>
</tr>
</table>


# 第二章：文本处理
本notebook需要使用的库和对应版本

In [2]:
from importlib.metadata import version

print("torch version: ", version('torch'))
print("tiktoken version: ", version('tiktoken'))


torch version:  2.2.1+cu118
tiktoken version:  0.6.0


- 本章介绍数据准备和采样，为大语言模型(LLM)的输入数据做好"准备"

<p align="center">
<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/01.webp?timestamp=1" width="500px">
</p>

## 2.1 理解词嵌入
- 本节没有代码
- 存在各种形式的嵌入，如视频嵌入、音频嵌入等，本书聚焦在文本嵌入。对于嵌入/embeddings可以理解为一种可表示原始数据对象的向量数据，因为训练过程就是一个计算的过程，需要将各类训练数据转换为可计算的形式。

<p align="center">
<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/02.webp" width="500px">
</p>

- LLMs在高维空间中与嵌入交互，如上千个维度
- 人类只能观测到1、2、3维空间，所以不能将高维空间可视化，但下图中二维空间可以大致表明嵌入空间的概念

<p align="center">
  <img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/03.webp" width="300px">
</p>

## 2.2 文本分词
- 本节对文本进行分词，或者理解为标记化处理，即将一段文本序列分解为更小的单元，如单个词或标点符号。下图中就是将一个句子按单词进行分词，分词后每个单词对应一个id，可以理解为该单词在整个vocab中的索引，也可以理解为分类任务中的一个类别标签，因为时数值，可以通过embedding层转换为嵌入向量

<p align="center">
  <img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/04.webp" width="300px">
</p>

- 加载准备处理的数据
- [The Verdict by Edith Wharton](https://en.wikisource.org/wiki/The_Verdict)是一个公开、无版权的短篇小说

In [ ]:
import os
import urllib.request

if not os.path.exists("the-verdict.txt"):
    url = ("https://raw.githubusercontent.com/rasbt/"
           "LLMs-from-scratch/main/ch02/01_main-chapter-code/"
           "the-verdict.txt")
    file_path = "the-verdict.txt"
    urllib.request.urlretrieve(url, file_path)  # 下载文件到指定地点

In [10]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

print("Total number of characters: ", len(raw_text))
print(raw_text[:99])

Total number of characters:  20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


- 目标是对文本进行分词和嵌入化，以其可作为LLMs的输入
- 基于一些简单的示例文本开发一个简单的分词器，之后可以应用到上面的文本中
- 以下正则表达式将按空白字符进行分割

In [4]:
import re

text = "Hello, world. This, is a test."
result = re.split(r'(\s)', text)

print(result)

['Hello,', ' ', 'world.', ' ', 'This,', ' ', 'is', ' ', 'a', ' ', 'test.']


- 不仅想按空白字符分割，还想按逗号和句号分割，所以让修改正则表达式以实现这一点

In [5]:
result = re.split(r'([,.]|\s)', text)

print(result)

['Hello', ',', '', ' ', 'world', '.', '', ' ', 'This', ',', '', ' ', 'is', ' ', 'a', ' ', 'test', '.', '']


- 上述实现会产生控制符，将其删除

In [6]:
# 消除每个元素前后的空字符，然后将所有为空字符的元素过滤掉
result = [item for item in result if item.strip()]
print(result)

['Hello', ',', 'world', '.', 'This', ',', 'is', 'a', 'test', '.']


- 上述实现不错，但是还要处理如句号、问号等标点符号

In [7]:
text = "Hello, world. Is this-- a test?"

result = re.split(r'([,.:;?_!"()\']|--|\s)', text)
result = [item.strip() for item in result if item.strip()]
print(result)

['Hello', ',', 'world', '.', 'Is', 'this', '--', 'a', 'test', '?']


- 上述实现的分词方式已经可以应用到原始文本了

<p align="center">
  <img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/05.webp" width="350px">
</p>

In [12]:
preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text)
preprocessed = [item.strip() for item in preprocessed if item.strip()]
print(preprocessed[:30])

['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '--', 'though', 'a', 'good', 'fellow', 'enough', '--', 'so', 'it', 'was', 'no', 'great', 'surprise', 'to', 'me', 'to', 'hear', 'that', ',', 'in']


- 查看所有tokens的数量

In [13]:
print(len(preprocessed))

4690


## 2.3 将tokens隐射为token IDs
- tokens转换为token IDs便于通过嵌入层进行计算

<p align="center">
<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/06.webp" width="500px">
</p>

- 基于从raw text中提取的tokens可以创建一个有所有唯一的token组成的词语表/vocab

In [14]:
all_words = sorted(set(preprocessed))
vocab_size = len(all_words)

print(vocab_size)

1130


- 构建一个key为token，value为id数值的字典，即一个便于将token隐射到id的映射关系

In [15]:
vocab = {token:integer for integer,token in enumerate(all_words)}

- 以下是词汇表中的前五十个实体

In [16]:
for i, item in enumerate(vocab.items()):
    print(item)
    if i >= 50:
        break

('!', 0)
('"', 1)
("'", 2)
('(', 3)
(')', 4)
(',', 5)
('--', 6)
('.', 7)
(':', 8)
(';', 9)
('?', 10)
('A', 11)
('Ah', 12)
('Among', 13)
('And', 14)
('Are', 15)
('Arrt', 16)
('As', 17)
('At', 18)
('Be', 19)
('Begin', 20)
('Burlington', 21)
('But', 22)
('By', 23)
('Carlo', 24)
('Chicago', 25)
('Claude', 26)
('Come', 27)
('Croft', 28)
('Destroyed', 29)
('Devonshire', 30)
('Don', 31)
('Dubarry', 32)
('Emperors', 33)
('Florence', 34)
('For', 35)
('Gallery', 36)
('Gideon', 37)
('Gisburn', 38)
('Gisburns', 39)
('Grafton', 40)
('Greek', 41)
('Grindle', 42)
('Grindles', 43)
('HAD', 44)
('Had', 45)
('Hang', 46)
('Has', 47)
('He', 48)
('Her', 49)
('Hermia', 50)


- 下图阐述了使用一个小的词汇表对一段短文进行分词的过程
<p align="center">
<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/07.webp?123" width="500px">
</p>


- 将上述关于分词的所有内容封装为一个类

In [18]:
class SimpleTokenizerV1:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i:s for s,i in vocab.items()}
    
    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
                                
        preprocessed = [
            item.strip() for item in preprocessed if item.strip()
        ]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids
        
    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        # 替换指定标点符号前的空格
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
        return text

- encode方法将文本转换为token IDs
- decode方法将token IDs转换会文本
<p align="center">
<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/08.webp?123" width="500px">
</p>

- 可使用分词器将文本编码为数值序列
- 数字序列进行嵌入化后作为LLM的输入

In [21]:
tokenizer = SimpleTokenizerV1(vocab)

text = """
           "It's the last he painted, you know," 
           Mrs. Gisburn said with pardonable pride.
       """
ids = tokenizer.encode(text)
print(ids)

[1, 56, 2, 850, 988, 602, 533, 746, 5, 1126, 596, 5, 1, 67, 7, 38, 851, 1108, 754, 793, 7]


- 将数值序列转换为文本

In [22]:
tokenizer.decode(ids)

'" It\' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.'

In [23]:
tokenizer.decode(tokenizer.encode(text))

'" It\' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.'

## 2.4 添加特殊上下文tokens
- 在文本中可能会存在一些不知道的词汇，也需要一些特殊词汇表示文本结束等，就需要设置一些特殊tokens；下图中的|unk|、|endoftext|就是分别表示不知道和文本结束的特俗tokens
<p align="center">
<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/09.webp?123" width="500px">
</p>